In [ ]:
# Interpolation and calculation of Drought Indices using precipitation data, hotspots and trends

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, ExpSineSquared, ConstantKernel, RBF, Matern, RationalQuadratic

In [ ]:
import time as time2
import os
# Interpolate spatially using GaussianProcessRegressor (modified Kriging)
# create a grid of latitude and longitude coordinates
lat_grid = np.linspace(55.3376, 69.1763, 100) 
lon_grid = np.linspace(11, 24.113, 100)
nx = len(lon_grid)
ny = len(lat_grid)
times=df_all_mon.ref.unique()
#datesout=pd.to_datetime(times)
dataout=np.zeros((len(times),ny,nx))
#dataout=np.load('dataout_16000.npy')
out_rsquared=[]
in_rsquared=[]

t=0

merged_df=pd.merge(df_stations[['id', 'latitude', 'longitude']], df_all_mon, left_on='id', right_on='station_id')
merged_df=merged_df.sort_values('ref', ascending=True)
start = time2.time()
for n in range(len(times)):
    time=times[n]
    temp_subset = merged_df[merged_df.ref==time]
    # Split data into testing and training sets
    merged_train = temp_subset[['latitude','longitude']]
    coords_train, coords_test, value_train, value_test = train_test_split(merged_train, temp_subset['value'], test_size = 0.20, random_state = 42)
    coords_train_wgs = [np.array(xy) for xy in zip(coords_train.longitude.values, coords_train.latitude.values)]
    coords_test_wgs = [np.array(xy) for xy in zip(coords_test.longitude.values, coords_test.latitude.values)]
    XX_sk_krig, YY_sk_krig = np.mgrid[lon_grid.min():lon_grid.max():100j, lat_grid.min():lat_grid.max():100j]
    # Create 2-D array of the coordinates (paired) of each cell in the mesh grid
    positions_sk_krig = np.vstack([XX_sk_krig.ravel(), YY_sk_krig.ravel()]).T
    kernel2= RationalQuadratic()
    kernel= RBF()

    # Define the hyperparameters
    length_scale = 0.1
    alpha = 0.1

    try:
        gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=3)
        # Fit kernel density estimator to coordinates and values
        gp.fit(coords_train_wgs, value_train)
    except:
        gp = GaussianProcessRegressor(kernel=kernel2, n_restarts_optimizer=3)
        # Fit kernel density estimator to coordinates and values
        gp.fit(coords_train_wgs, value_train)
        print('error')
    # Evaluate the model on coordinate pairs
    Z_sk_krig, std_prediction = gp.predict(positions_sk_krig, return_std=True)
    # Reshape the data to fit mesh grid
    interp_temps = Z_sk_krig.reshape(XX_sk_krig.shape)
    # Generate in-sample R^2
    in_r_squared_sk_krig = gp.score(coords_train_wgs, value_train)
    print("Scikit-Learn Kriging in-sample r-squared: {}".format(round(in_r_squared_sk_krig, 2)))
    # Generate out-of-sample R^2
    out_r_squared_sk_krig = gp.score(coords_test_wgs, value_test)
    print("Scikit-Learn Kriging out-of-sample r-squared: {}".format(round(out_r_squared_sk_krig, 2)))
    Z_sk_krig[Z_sk_krig<0]=0
    Z_sk_krig[Z_sk_krig>np.nanmax(value_train)]=np.nanmax(value_train)
    # Check for stable results
    #if np.nanmax(interp_temps)<=upper_limit and np.nanmin(interp_temps)>=lower_limit:
    if out_r_squared_sk_krig>-1:
        in_rsquared.append(in_r_squared_sk_krig)
        out_rsquared.append(out_r_squared_sk_krig)
        dataout[t,:]=interp_temps
        end = time2.time()
        print('Time elapsed (s): '+ str(end - start))
        #break
    else:
        #dataout[t,:]=np.nan
        print('Low correlation, trying again...')
        kernel= RationalQuadratic()
        gp = GaussianProcessRegressor(kernel=kernel, alpha=0.1, n_restarts_optimizer=5)
        gp.fit(coords_train_wgs, value_train)
        Z_sk_krig, std_prediction = gp.predict(positions_sk_krig, return_std=True)
        Z_sk_krig[Z_sk_krig<0]=0
        Z_sk_krig[Z_sk_krig>np.nanmax(value_train)]=np.nanmax(value_train)
        interp_temps = Z_sk_krig.reshape(XX_sk_krig.shape)
        in_r_squared_sk_krig = gp.score(coords_train_wgs, value_train)
        print("Scikit-Learn Kriging in-sample r-squared: {}".format(round(in_r_squared_sk_krig, 2)))
        out_r_squared_sk_krig = gp.score(coords_test_wgs, value_test)
        print("Scikit-Learn Kriging out-of-sample r-squared: {}".format(round(out_r_squared_sk_krig, 2)))
        #kernel= RationalQuadratic(0.1) * (DotProduct(0.1))**2
        if np.nanmax(interp_temps)<=upper_limit and np.nanmin(interp_temps)>=lower_limit:
            dataout[t,:]=interp_temps 
            in_rsquared.append(in_r_squared_sk_krig)
            out_rsquared.append(out_r_squared_sk_krig)
        else:
            #dataout[t,:]=np.nan
            print('error')
            break
    print(time)
    print(t)
    print('Mean interpolated values: '+str(np.mean(dataout[t,:])))
    t=t+1
print(np.mean(in_rsquared))
print(np.mean(out_rsquared))

In [ ]:
# Mask out values outside country boundaries
from shapely.geometry import Polygon, Point
from cartopy.io import shapereader
import geopandas
def rect_from_bound(xmin, xmax, ymin, ymax):
    """Returns list of (x,y)'s for a rectangle"""
    xs = [xmax, xmin, xmin, xmax, xmax]
    ys = [ymax, ymax, ymin, ymin, ymax]
    return [(x, y) for x, y in zip(xs, ys)]
# Mask from natural earth (more below)
# request data for use by geopandas
resolution = '50m'
category = 'cultural'
name = 'admin_0_countries'

shpfilename = shapereader.natural_earth(resolution, category, name)
df = geopandas.read_file(shpfilename)

# get geometry and polygon of a country
poly = [df.loc[df['ADMIN'] == 'Sweden']['geometry'].values[0]]
exts = [min(lon_grid)-1, max(lon_grid)+1, min(lat_grid)-1, max(lat_grid)+1]
msk = Polygon(rect_from_bound(*exts)).difference( poly[0].simplify(0.01) )
# create boolean mask where True=inside polygon
mask=np.zeros((len(lat_grid),len(lon_grid)),dtype=bool)
for i, y in enumerate(lat_grid):
    for j, x in enumerate(lon_grid):
        mask[i,j]=Point(x,y).within(msk)


In [ ]:
# Clean interpolated data to remove values out of Sweden
times=df_all_mon.ref.unique()
for t in range(len(times)):
    dataout[t,:]=np.ma.array(dataout[t,:].T, mask = mask).filled(np.nan) # Transpose to correct plot for grid
    #t=t+1

In [ ]:
precip_mon.close()

In [ ]:
ds.close()

In [ ]:
# Save netcdf data (xarray)
import xarray as xr

# Create a 1d array for the time axis
times=df_all_mon.ref.unique()
# Create a DataArray from the dataout numpy array, with the time axis as the first dimension
da = xr.DataArray(dataout, dims=('time', 'lat', 'lon'), coords={'time': times})
da['lat']=lat_grid
da['lon']=lon_grid
# Convert the DataArray to a Dataset
ds = da.to_dataset(name='precipitation')

# Save the Dataset to a netCDF file
ds.to_netcdf('precip-12mon.nc')

In [ ]:
#precip_mon = xr.open_dataset('precip-1mon.nc')
precip_yr = xr.open_dataset('precip-12mon.nc')

In [ ]:
np.nanmax(precip_mon['precipitation'].values)

In [ ]:
# When did the maximum rainfall occured?
np.where(precip_mon['precipitation'].values==np.nanmax(precip_mon['precipitation'].values))

In [ ]:
precip['time'].values[13791]

In [ ]:
lat_grid[93], lon_grid[55]

In [ ]:
# Plot average precipitation rates
precip_values = precip_mon['precipitation'].values
# Mask the zero values in the array using numpy.where()
masked_values = np.where(precip_values != 0, precip_values, np.nan)
# Calculate the mean of the non-zero values using numpy.nanmean()
precip_mean = np.nanmean(masked_values, axis=0)
plot_map(precip_mean, 'Average Monthly Precipitation (mm)', 0, 5, 'imshow')

In [ ]:
precip_mon

In [ ]:
len(precip.time)

In [ ]:
f=365 # conversion for 12-monthly precipitation
precip = xr.open_dataset('precipitation.nc')#.round()
precip_rolling= precip.rolling(time=f, min_periods=f).sum()
# calculate daily rolling sums with a window size of f days
precip_rolling = precip.rolling(time=f, min_periods=1).sum()
precip_yr = precip_rolling
#precip_yr = xr.open_dataset('precip-12mon.nc')

In [ ]:
import numpy as np
lat_grid = np.linspace(55.3376, 69.1763, 100) 
lon_grid = np.linspace(11, 24.113, 100)

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import kstest, gamma
def spi_transform(H):
    #print(H)
    norm=[]
    c0= 2.515517
    c1= 0.802853
    c2= 0.010328
    d1= 1.432788
    d2= 0.189269
    d3= 0.001308
    for h in H:
        #print(h)
        if h<=0.5:
            t=np.sqrt(np.log(1/h**2))
            spi_h=-(t-(c0+c1*t+c2*t**2)/(1+d1*t+d2*t**2+d3*t**3))
        elif h>0.5: #and h<1:
            t=np.sqrt(np.log(1/((1-h)**2)))
            spi_h=+(t-(c0+c1*t+c2*t**2)/(1+d1*t+d2*t**2+d3*t**3))
        else:
            spi_h=np.nan
        norm.append(spi_h)
    return norm
#from standard_precip.spi import SPI
#from standard_precip.utils import plot_index
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]
# Test calculating SPI at one point
def SPI_point(lat, lon, precip_mon):
    #dist=gamma
    cutoff=0
    lat_near= find_nearest(precip_mon.lat.values, lat)
    lon_near= find_nearest(precip_mon.lon.values, lon)
    rf_data_point = pd.DataFrame(columns=['time', 'precipitation'])
    rf_data_point['precipitation'] = precip_mon.sel(lat=lat_near, lon=lon_near)['precipitation'].values
    rf_data_point['time'] = precip_mon.sel(lat=lat_near, lon=lon_near)['time'].values
    #print(rf_data_point.max())
    if rf_data_point['precipitation'].mean()>0: # check if grid point is inside Sweden
        #rf_data_point= rf_data_point[rf_data_point['daily precipitation']>=cutoff]
        df_spi_point_sort= rf_data_point.copy()
        df_spi_point_sort= df_spi_point_sort.sort_values(by='precipitation', ascending=True)
        #df_spi_point_sort= df_spi_point_sort[df_spi_point_sort['daily precipitation']>cutoff]
        # Plot histogram of monthly precipitation
        pre_dist= rf_data_point['precipitation'].values
        gamma_fit, cdf_dist, norm_ppf = fit_gamma(pre_dist, 0) 
        df_spi_point_sort['SPI']=norm_ppf
        df_spi_point_sort=df_spi_point_sort.sort_values(by='time', ascending=True)
    else:
        df_spi_point_sort = pd.DataFrame(columns=['SPI'])
        df_spi_point_sort['SPI'] = [np.nan for x in precip_mon.time.values]
    return df_spi_point_sort

In [ ]:
f=365 # SPEI12
tmax_mon= tmax_ds.rolling(time=f, min_periods=f).sum().resample(time='1M').first()
tmin_mon= tmin_ds.rolling(time=f, min_periods=f).sum().resample(time='1M').first()

In [ ]:
## Hargreaves (1982)
def hargreaves(Tmin, Tmax, date, latitude):
    # Computation of extra-terrestrial solar radiation
    doy = (date - np.datetime64(date.astype('datetime64[Y]'))) / np.timedelta64(1, 'D') + 1
    #doy = date.dayofyear
    dr = 1 + 0.033 * np.cos(2 * np.pi * doy/365) # Inverse relative distance Earth-Sun
    phi = np.pi / 180 *  latitude # Latitude in radians
    d = 0.409 * np.sin((2 * np.pi * doy/365) - 1.39) # Solar declination
    omega = np.arccos(-np.tan(phi) * np.tan(d)) # Sunset hour angle
    Gsc = 0.0820 # Solar constant
    Ra = 24 * 60 / np.pi * Gsc * dr * (omega * np.sin(phi) * np.sin(d) + np.cos(phi) * np.cos(d) * np.sin(omega))
    Tm = (Tmin + Tmax)/2 # temperature in celsius degrees
    PET = 0.0023 * Ra * (Tm + 17.8) * (Tmax - Tmin)**0.5
    Evap_m = PET / (2.45 * 1000) # output in m/d
    Evap_mm = Evap_m*1000 # convert to mm/d
    return Evap_mm 
def SPEI_point(lat, lon, precip_mon, tmax_data, tmin_data, time_daily, f, i, j):
    #dist=gamma
    cutoff=0
    lat_near= find_nearest(precip_mon.lat.values, lat)
    lon_near= find_nearest(precip_mon.lon.values, lon)
    rf_data_point = pd.DataFrame(columns=['time', 'precipitation'])
    rf_data_point['precipitation'] = precip_mon.sel(lat=lat_near, lon=lon_near)['precipitation'].values
    rf_data_point['time'] = precip_mon.sel(lat=lat_near, lon=lon_near)['time'].values
    #print(rf_data_point.max())
    if rf_data_point['precipitation'].mean()>0: # check if grid point is inside Sweden
        pet_daily=[]
        # Extract temperature and time data for grid point
        tmax_daily = tmax_data[:,i,j]
        tmin_daily = tmin_data[:,i,j]
        for i in range(len(time_daily)):
            Tmax=tmax_daily[i]
            Tmin=tmin_daily[i]
            date=time_daily[i]
            pet=hargreaves(Tmin, Tmax, date, lat_near)
            if np.isnan(pet):
                pet_daily.append(0)
            else:
                pet_daily.append(pet)
        pet_df=pd.DataFrame(columns=('time','pet'))
        pet_df['time']=pd.to_datetime(time_daily)
        pet_df['pet']=pet_daily
        pet_mon=pet_df.set_index('time').rolling(f, min_periods=f).sum().resample('1M').first()
        #pet_mon=pet_mon.fillna(0)
        rf_data_point['pre_excess']= rf_data_point['precipitation'].values-pet_mon['pet'].values
        df_spi_point_sort= rf_data_point.copy()
        df_spi_point_sort= df_spi_point_sort.sort_values(by='pre_excess', ascending=True)
        # Plot histogram of monthly precipitation excess
        pre_dist= rf_data_point['pre_excess'].values
        #print(pet_mon['pet'])
        pre_dist = pre_dist[~np.isnan(pre_dist)]
        pdf_dist, cdf_dist, norm_ppf = fit_pearson3(pre_dist) 
        nan_array = np.empty(len(df_spi_point_sort.index) - len(norm_ppf))
        nan_array[:] = np.nan
        norm_ppf = np.concatenate((nan_array, norm_ppf))
        df_spi_point_sort['SPEI']=norm_ppf
        df_spi_point_sort=df_spi_point_sort.sort_values(by='time', ascending=True)
    else:
        df_spi_point_sort = pd.DataFrame(columns=['SPEI'])
        df_spi_point_sort['SPEI'] = [np.nan for x in precip_mon.time.values]
    return df_spi_point_sort

In [ ]:
lat_near= lat_grid[j]
lon_near= lon_grid[i]
rf_data_point = pd.DataFrame(columns=['time', 'precipitation'])
rf_data_point['precipitation'] = precip_mon.sel(lat=lat_near, lon=lon_near)['precipitation'].values

In [ ]:
# Calculate Potential Evapotranspiration at all points (not used)
import numpy as np
import pandas as pd
# Import data
tmax_data = tmax_ds['maximum daily air temperature'].values
tmin_data = tmin_ds['maximum daily air temperature'].values
time_data = tmax_ds['time'].values

# Define grid
lat_grid = np.linspace(55.3376, 69.1763, 100)
lon_grid = np.linspace(11, 24.113, 100)

# Initialize PET array
pet_mon = np.zeros((len(tmax_ds.time.values), 100, 100))

# Loop over grid points
for i in range(len(lat_grid)):
    for j in range(len(lon_grid)):
        print(i,j)
        # Extract temperature and time data for grid point
        tmax_daily = tmax_data[:,i,j]
        tmin_daily = tmin_data[:,i,j]
        time_daily = time_data[:,i,j]
        
        # Check if grid point is inside Sweden
        if np.mean(tmax_daily) > 0:
            # Compute daily PET values
            pet_daily = np.array([hargreaves(Tmin, Tmax, date, lat_near) for Tmin, Tmax, date in zip(tmin_daily, tmax_daily, time_daily)])
            
            # Convert to DataFrame and resample to monthly values
            pet_df = pd.DataFrame({'time': time_daily, 'pet': pet_daily})
            pet_df['time'] = pd.to_datetime(pet_df['time'])
            pet_mon[:, i, j] = pet_df.set_index('time').rolling(f, min_periods=f).sum().resample('1M').first()['pet'].values
        else:
            pet_mon[:, i, j] = np.nan


In [ ]:
# Compute X-month SPEI at all points 
import xarray as xr
import pandas as pd
import numpy as np
precip_per=precip_yr
#tmax_per=tmax_mon
#tmin_per=tmin_mon
# Import data
tmax_per = tmax_ds['maximum daily air temperature'].values
tmin_per = tmin_ds['maximum daily air temperature'].values
time_per = tmax_ds['time'].values
f=365
# Calculate SPEI for all points
#SPEI_data = np.zeros((len(precip_per.time.values), 100, 100))
lat_grid = np.linspace(55.3376, 69.1763, 100) 
lon_grid = np.linspace(11, 24.113, 100)
for i in range(len(lat_grid)):
    for j in range(len(lon_grid)):
        print(i,j)
        df_spi_point = SPEI_point(lat_grid[i], lon_grid[j], precip_per, tmax_per, tmin_per, time_per, f, i, j) 
        SPEI_data[:,i,j] = df_spi_point['SPEI'].values

In [ ]:
# Compute X-month SPI at all points 
import xarray as xr
import pandas as pd
import numpy as np
precip_per=precip_yr
# Calculate SPI for all points
SPI_data = np.zeros((len(precip_per.time.values), 100, 100))
lat_grid = np.linspace(55.3376, 69.1763, 100) 
lon_grid = np.linspace(11, 24.113, 100)
for i in range(len(lat_grid)):
    for j in range(len(lon_grid)):
        print(i,j)
        df_spi_point = SPI_point(lat_grid[i], lon_grid[j], precip_per) 
        SPI_data[:,i,j] = df_spi_point['SPI'].values

In [ ]:
# Save SPI/SPEI data to file
import xarray as xr

# Create a 1d array for the time axis
times = precip_per.time.values

# Create a DataArray from the dataout numpy array, with the time axis as the first dimension
da = xr.DataArray(SPEI_data, dims=('time', 'lat', 'lon'), coords={'time': times})
da['lat']=lat_grid
da['lon']=lon_grid
# Convert the DataArray to a Dataset
ds = da.to_dataset(name='SPEI')

# Save the Dataset to a netCDF file
#ds.to_netcdf('SPEI112.nc')

In [ ]:
lat= 59.3293
lon= 18.0686
lat_near= find_nearest(precip_mon.lat.values, lat)
lon_near= find_nearest(precip_mon.lon.values, lon)
rf_data_point = pd.DataFrame(columns=['time', 'daily precipitation'])
rf_data_point['daily precipitation'] = precip_mon.sel(lat=lat_near, lon=lon_near)['daily precipitation'].values
rf_data_point['time'] = precip_mon.sel(lat=lat_near, lon=lon_near)['time'].values
#rf_data_point= rf_data_point[rf_data_point['daily precipitation']>=cutoff]
df_spi_point_sort= rf_data_point.copy()
df_spi_point_sort= df_spi_point_sort.sort_values(by='daily precipitation', ascending=True)
#df_spi_point_sort= df_spi_point_sort[df_spi_point_sort['daily precipitation']>=cutoff]

# Plot histogram of monthly precipitation
pre_dist= rf_data_point['daily precipitation'].values
#pre_dist[np.isnan(pre_dist)] = 0
precipitation= pre_dist
gamma_fit, beta, alpha, A = fit_gamma(precipitation, 0)

In [ ]:
import numpy as np
import math
import scipy.stats as scs
from scipy.stats import gamma as gamma_scs
import scipy.special

def fit_gamma(X, cutoff):
    """
    Fits a gamma distribution to a dataset X using the method described in the original paper.

    Parameters:
    X (numpy array): The dataset to fit the gamma distribution to.
    cutoff (numeric): The minimum value in X to include in the fit.

    Returns:
    gamma (numpy array): The probability density function values of the fitted gamma distribution.
    beta (numeric): The beta parameter of the fitted gamma distribution.
    alpha (numeric): The alpha parameter of the fitted gamma distribution.
    A (numeric): The A value used in the calculation of alpha and beta.
    """

    # Sort the data and remove NaNs and values below the cutoff
    X = np.sort(X)
    Y = X.copy()
    X = X[~np.isnan(X)]
    X = X[X > cutoff]

    # Calculate the values of alpha and beta
    n = len(X)
    A = np.log(np.mean(X)) - np.sum(np.log(X)) / n
    alpha = 1 / (4 * A) * (1 + np.sqrt(1 + 4 * A / 3))
    beta = np.mean(X) / alpha

    # Calculate the probability density function values for each data point
    n = len(Y)
    gamma = np.zeros(n)
    for i, x in enumerate(Y):
        if x>0:
            gamma[i] = 1 / (beta ** alpha * scipy.special.gamma(alpha)) * x ** (alpha - 1) * np.exp(-x / beta)
        else:
            gamma[i] = np.nan
    cdf_dist = gamma_scs.cdf(np.sort(Y), a=alpha, loc=0, scale=beta)
    q=len(Y[Y==0]) / len(Y) # Number of zeroes divided by number of total observations
    print(q)
    H = q + (1 - q) * cdf_dist
    norm_ppf=spi_transform(H)
    #norm_ppf = scs.norm.ppf(H) # same results as above but requires extra line below
    #norm_ppf[np.isinf(norm_ppf)] = np.nan
    
    return gamma, cdf_dist, norm_ppf

In [ ]:
import numpy as np
import math
import scipy.stats as scs
from scipy.stats import fisk, pearson3, lognorm

def fit_pearson3(X):
    """
    Fits a log-logistic distribution to a dataset X using the method described in the original paper.

    Parameters:
    X (numpy array): The dataset to fit the log-logistic distribution to.
    cutoff (numeric): The minimum value in X to include in the fit.

    Returns:
    gamma (numpy array): The probability density function values of the fitted log-logistic distribution.
    alpha (numeric): The alpha parameter of the fitted log-logistic distribution.
    beta (numeric): The beta parameter of the fitted log-logistic distribution.
    A (numeric): The A value used in the calculation of alpha and beta.
    """

    # Sort the data and remove NaNs and values below the cutoff
    X = X[~np.isnan(X)]
    X = np.sort(X)
    #print(np.max(X))
    #Y = X.copy()
    
    #X = X[X > cutoff]
    # Calculate the values of moments
    n = len(X)
    w0=0
    w1=0
    w2=0
    for i in range(n):
        Fi=(i-0.35)/n
        w0=w0+1/n*(1-Fi)**0*X[i]
        w1=w1+1/n*(1-Fi)**1*X[i]
        w2=w2+1/n*(1-Fi)**2*X[i]
    # Calculate the values of alpha, beta and gamma
    beta = (2*w1-w0)/(6*w1-w0-6*w2)
    alpha = ((w0-2*w1)*beta)/(math.gamma(1+1/beta)*math.gamma(1-1/beta))
    gamma = w0-alpha*(math.gamma(1+1/beta)*math.gamma(1-1/beta))
    # Calculate the probability density function values for each data point
    pdf_dist = np.zeros(n)
    cdf_dist = np.zeros(n)
    for i, x in enumerate(X):
        pdf_dist[i] = (beta/alpha)*((x-gamma)/alpha)**(beta-1)*(1+((x-gamma)/alpha)**beta)**(-2)
        cdf_dist[i] = (1+(alpha/(x-gamma))**beta)**(-1)
    
    norm_ppf=spi_transform(cdf_dist)

    return pdf_dist, cdf_dist, norm_ppf


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.collections import LineCollection
def plot_spi(df_spi_point_sort, option):
    # get the SPI/SPEI values
    spi_values = df_spi_point_sort[option]
    # Plot SPI over time
    fig, ax = plt.subplots(figsize=(8, 6))
    y1positive=np.array(spi_values)>=0
    y1negative = np.array(spi_values)<=0
    plt.fill_between(df_spi_point_sort['time'], spi_values, y2=0,where=y1positive,
    color='blue',alpha=0.5,interpolate=False)
    plt.fill_between(df_spi_point_sort['time'], spi_values, y2=0,where=y1negative,
    color='red',alpha=0.5,interpolate=False)
    #colors = ['r' if spi < 0 else 'b' for spi in spi_values]
    #plt.plot(df_spi_point_sort['time'], spi_values, label='SPI', color=colors)
    plt.axhline(0, color='k', linestyle='--')  # add a horizontal line at y=0
    plt.title(option+' Timeseries')
    plt.xlabel('Time')
    plt.ylabel(option)
    plt.ylim([-4,4])
    plt.legend()
    plt.show()
    


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#from scipy.stats import invgauss, kstest, gamma, pearson3, genextreme, norm, expon, weibull_max, gumbel_l, invgamma, lognorm, rayleigh, skewnorm 
import scipy.stats as scs
from scipy.stats import gamma as gamma_scs
def ecdf(data):
    # Compute ECDF
    x = np.sort(data)
    n = x.size
    y = np.arange(1, n+1) / n
    return(x,y)
def plot_distributions(lat, lon, precip_mon, dist, n_bins, cutoff, option, f):
    lat_near= find_nearest(precip_mon.lat.values, lat)
    lon_near= find_nearest(precip_mon.lon.values, lon)
    rf_data_point = pd.DataFrame(columns=['time', 'precipitation'])
    rf_data_point['precipitation'] = precip_mon.sel(lat=lat_near, lon=lon_near)['precipitation'].values
    rf_data_point['time'] = precip_mon.sel(lat=lat_near, lon=lon_near)['time'].values
    #rf_data_point= rf_data_point[rf_data_point['daily precipitation']>=cutoff]
    df_spi_point_sort= rf_data_point.copy()
    df_spi_point_sort= df_spi_point_sort.sort_values(by='precipitation', ascending=True)
    #df_spi_point_sort= df_spi_point_sort[df_spi_point_sort['daily precipitation']>cutoff]
    pre_dist= rf_data_point['precipitation'].values
    #pre_dist[np.isnan(pre_dist)] = 0
    if option=='SPI': # results for SPI
        # Plot histogram of monthly precipitation
        precipitation= pre_dist[pre_dist>cutoff] # ignore dry days
        #dates=dates[np.where(pre_dist>0)]
        # Compute histogram of monthly precipitation and fit a gamma distribution
        hist, bin_edges = np.histogram(precipitation, bins=n_bins, density=False)
        bin_centers = (bin_edges[:-1] + bin_edges[1:])/2
        total_count = len(precipitation)
        bin_width = bin_edges[1] - bin_edges[0]
        gamma_fit, cdf_dist, norm_ppf = fit_gamma(pre_dist, 0) 
        #gamma_fit2= gamma_fit[gamma_fit>0]
        # Generate x values for gamma distribution PDF
        #x = np.linspace(0, np.max(precipitation), len(df_spi_point_sort))
        #x = np.linspace(0, np.max(precipitation), len(gamma_fit))
        # Plot histogram of monthly precipitation and fitted gamma distribution
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.hist(precipitation, bins=n_bins, density=False, alpha=0.5, edgecolor='black')
        ax.plot(np.sort(pre_dist), gamma_fit* total_count * bin_width, color='red', label='Fitted Gamma Distribution')
        #ax.scatter(precipitation, np.zeros_like(precipitation) + 0.01, alpha=0.5, label='Empirical data')
        #ax.plot(x, dist.pdf(x, *params)* total_count * bin_width, color='red', label='Fitted Distribution')
        ax.set_title('Precipitation at point near Stockholm')
        ax.set_xlabel('Precipitation (mm)')
        ax.set_ylabel('Frequency')
        ax.set_xlim([0, np.max(precipitation)])
        #ax.set_ylim([0, 300])
        ax.legend()
        fig.show()
        # Compute cumulative distribution function for fitted gamma distribution
        #cdf_dist = dist.cdf(x, *params)
        ##cdf_dist = gamma.cdf(np.sort(pre_dist), a=alpha, loc=0, scale=beta)
        #x = np.linspace(0, np.max(precipitation), len(df_spi_point_sort))
        #x = np.linspace(0, np.max(precipitation), len(cdf_dist))
        fig, ax = plt.subplots(figsize=(6, 6))
        x_emp, y_emp=ecdf(precipitation)
        plt.plot(x_emp, y_emp, '.', color='black', label='Observed Points')
        plt.plot(np.sort(pre_dist), np.sort(cdf_dist), color='red', label='Fitted Distribution')
        plt.title('Cumulative Probability Function')
        plt.xlabel('Precipitation (mm)')
        plt.ylabel('Cumulative Probability')
        plt.legend()
        plt.show()
        # Plot cumulative distribution function for fitted gamma distribution and SPI values
        fig, ax = plt.subplots(figsize=(6, 6))
        plt.plot(norm_ppf, cdf_dist, color='red', label='Transformed Distribution')
        #plt.plot(spi, np.arange(len(spi))/float(len(spi)), '.', color='black', label='Observed Points')
        plt.title('Standardized Precipitation Index')
        plt.xlabel('Standardized Precipitation Index')
        plt.ylabel('Cumulative Probability')
        plt.xlim([-4,4])
        plt.legend()
        plt.show()
        df_spi_point_sort['SPI']=norm_ppf
        df_spi_point_sort=df_spi_point_sort.sort_values(by='time', ascending=True)
    elif option=='SPEI': # results for SPEI
        pet_daily=[]
        tmax_daily=tmax_ds.sel(lat=lat_near, lon=lon_near)['maximum daily air temperature'].values
        tmin_daily=tmin_ds.sel(lat=lat_near, lon=lon_near)['maximum daily air temperature'].values
        time_daily=tmax_ds.sel(lat=lat_near, lon=lon_near)['time'].values
        for i in range(len(time_daily)):
            Tmax=tmax_daily[i]
            Tmin=tmin_daily[i]
            date=time_daily[i]
            #print(type(date))
            pet_daily.append(hargreaves(Tmin, Tmax, date, lat_near))
        pet_df=pd.DataFrame(columns=('time','pet'))
        pet_df['time']=pd.to_datetime(time_daily)
        pet_df['pet']=pet_daily
        pet_mon=pet_df.set_index('time').rolling(f, min_periods=f).sum().resample('1M').first()
        #pet_mon=pet_mon.fillna(0)
        rf_data_point['pre_excess']= rf_data_point['precipitation'].values-pet_mon['pet'].values
        #df_spi_point_sort= df_spi_point_sort.sort_values(by='pre_excess', ascending=True)
        # Plot histogram of monthly precipitation excess
        pre_dist= rf_data_point['pre_excess'].values
        #print(pet_mon['pet'])
        pre_dist = pre_dist[~np.isnan(pre_dist)]
        pdf_dist, cdf_dist, norm_ppf = fit_pearson3(pre_dist) 
        #pre_dist2 = pre_dist[~np.isnan(pre_dist)]
        # Plot histogram of monthly precipitation excess
        # Compute histogram of monthly precipitation and fit a gamma distribution
        #norm_ppf=norm_ppf[~np.isnan(norm_ppf)]
        hist, bin_edges = np.histogram(pre_dist, bins=n_bins, density=False)
        bin_centers = (bin_edges[:-1] + bin_edges[1:])/2
        total_count = len(pre_dist)
        bin_width = bin_edges[1] - bin_edges[0]
        # Generate x values for gamma distribution PDF
        # Plot histogram of monthly precipitation and fitted gamma distribution
        nan_array2 = np.empty(len(pdf_dist) - len(pre_dist))
        nan_array2[:] = np.nan
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.hist(pre_dist, bins=n_bins, density=False, alpha=0.5, edgecolor='black')
        ax.plot(np.sort(pre_dist), pdf_dist* total_count * bin_width, color='red', label='Fitted Log-logistic distribution')
        ax.set_title('Precipitation excess at point near Stockholm')
        ax.set_xlabel('Precipitation excess (mm)')
        ax.set_ylabel('Frequency')
        #ax.set_xlim([0, np.max(pre_dist)])
        #ax.set_ylim([0, 300])
        ax.legend()
        fig.show()
        fig, ax = plt.subplots(figsize=(6, 6))
        x_emp, y_emp=ecdf(pre_dist)
        plt.plot(x_emp, y_emp, '.', color='black', label='Observed Points')
        #pre_dist = np.concatenate((nan_array, pre_dist))
        plt.plot(np.sort(pre_dist), np.sort(cdf_dist), color='red', label='Fitted Distribution')
        plt.title('Cumulative Probability Function')
        plt.xlabel('Precipitation Excess (mm)')
        plt.ylabel('Cumulative Probability')
        plt.legend()
        plt.show()
        # Plot cumulative distribution function for fitted gamma distribution and SPI values
        fig, ax = plt.subplots(figsize=(6, 6))
        plt.plot(norm_ppf, cdf_dist, color='red', label='Transformed Distribution')
        #plt.plot(spi, np.arange(len(spi))/float(len(spi)), '.', color='black', label='Observed Points')
        plt.title('Standardized Precipitation Evapotranspiration Index')
        plt.xlabel('SPEI')
        plt.ylabel('Cumulative Probability')
        plt.xlim([-4,4])
        plt.legend()
        plt.show()
        nan_array = np.empty(len(df_spi_point_sort.index) - len(norm_ppf))
        nan_array[:] = np.nan
        norm_ppf = np.concatenate((nan_array, norm_ppf))
        #pdf_fit = np.concatenate((nan_array, pdf_fit))
        #cdf_dist = np.concatenate((nan_array, cdf_dist))
        df_spi_point_sort['SPEI']=norm_ppf
        df_spi_point_sort=df_spi_point_sort.sort_values(by='time', ascending=True)
    # Plot SPI over time
    plot_spi(df_spi_point_sort, option)
    return df_spi_point_sort
    

In [ ]:
from shapely.geometry import Polygon
from cartopy.io import shapereader
import cartopy.io.img_tiles as cimgt
import cartopy.crs as ccrs
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from scipy.interpolate import griddata
import matplotlib.colors as colors
from matplotlib.colors import LinearSegmentedColormap, BoundaryNorm
import matplotlib.ticker as ticker
plt.rcdefaults()
resolution = '50m'
category = 'cultural'
name = 'admin_0_countries'

shpfilename = shapereader.natural_earth(resolution, category, name)

df = gpd.read_file(shpfilename)

def rect_from_bound(xmin, xmax, ymin, ymax):
    """Returns list of (x,y)'s for a rectangle"""
    xs = [xmax, xmin, xmin, xmax, xmax]
    ys = [ymax, ymax, ymin, ymin, ymax]
    return [(x, y) for x, y in zip(xs, ys)]

def plot_map(interp_temps, title, label, lower_bound, upper_bound, vcenter, option, cmap_type, save, filename):
    # request data for use by geopandas
    resolution = '10m'
    category = 'cultural'
    name = 'admin_0_countries'
    data=interp_temps.copy()
    shpfilename = shapereader.natural_earth(resolution, category, name)
    df = geopandas.read_file(shpfilename)

    # get geometry of a country
    poly = [df.loc[df['ADMIN'] == 'Sweden']['geometry'].values[0]]

    stamen_terrain = cimgt.Stamen('terrain-background')
    # define the colormap
    if cmap_type=='SPI_cmap':
        cmap = colors.ListedColormap(['red', 'orange', 'yellow', 'white', 'lavender', 'plum', 'purple'])
        #define the boundaries of the SPI values for each color in the colormap
        #bounds = [-np.inf, -2.0, -1.5, -1.0, 1.0, 1.5, 2.0, np.inf]
        # create a norm object to map SPI values to colors
        #norm = colors.BoundaryNorm(bounds, cmap.N)
    elif cmap_type=='inverted':
        cmap='coolwarm_r' #'RdBu'
    else:
        cmap='coolwarm' #'RdBu_r'

    # projections that involved
    st_proj = ccrs.PlateCarree()  #projection used by Stamen images
    ll_proj = ccrs.PlateCarree()  #CRS for raw long/lat

    # create fig and axes using intended projection
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(1, 1, 1, projection=st_proj)
    ax.add_geometries(poly, crs=ll_proj, facecolor='none', edgecolor='black')

    pad1 = .1  #padding, degrees unit
    exts = [poly[0].bounds[0] - pad1, poly[0].bounds[2] + pad1, poly[0].bounds[1] - pad1, poly[0].bounds[3] + pad1];
    ax.set_extent(exts, crs=ll_proj)

    # make a mask polygon by polygon's difference operation
    # base polygon is a rectangle, another polygon is simplified switzerland
    msk = Polygon(rect_from_bound(*exts)).difference( poly[0].simplify(0.01) )
    msk_stm  = st_proj.project_geometry (msk, ll_proj)  # project geometry to the projection used by stamen

    # get and plot Stamen images
    #ax.add_image(stamen_terrain, 8) # this requests image, and plot

    # plot the mask using semi-transparency (alpha=0.65) on the masked-out portion
    ax.add_geometries( msk_stm, st_proj, zorder=12, facecolor='white', edgecolor='k', alpha=1)

    #gl=ax.gridlines(draw_labels=True)
    #gl.xlabels_top = False
    #gl.ylabels_left = False
    

    if option=='imshow':
        data[data>upper_bound]=upper_bound
        data[data<lower_bound]=lower_bound
        if cmap_type=='SPI_cmap':
            bounds=[-3.0, -2.0, -1.5, -1.0, 1.0, 1.5, 2.0, 3.0]
            norm = colors.BoundaryNorm(bounds, cmap.N)
            im = ax.imshow(data, origin='lower', norm=norm,
                extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                cmap=cmap, zorder=11)
        else:
            # Create bounds vector from 10th to 90th percentile with median in the center
            # Calculate the distance between center, lower and upper bounds
            lb=lower_bound
            ub=upper_bound
            dist = vcenter - lb
            dist2 = ub - vcenter
            # Create bounds vector from 10th to 90th percentile with median closer to p10
            bounds = [lb, lb + 0.25*dist , lb + 0.4*dist, lb + 0.5*dist, lb + 0.7*dist, lb + 0.8*dist, lb + 0.9*dist, lb + dist, vcenter + 0.25*dist2, vcenter + 0.5*dist2, vcenter + 0.75*dist2, vcenter + 0.8*dist2, vcenter + 0.9*dist2, ub]
            im = ax.imshow(data, origin='lower', #norm=BoundaryNorm(bounds, ncolors=256),
                vmin= lb, vmax=ub,
                extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
                cmap=cmap, zorder=11)
    else:
        if cmap_type=='SPI_cmap':
            bounds=[-3.0, -2.0, -1.5, -1.0, 1.0, 1.5, 2.0, 3.0]
        else:
            bounds= np.linspace(lower_bound, upper_bound, num=12)
        data[data>upper_bound]=upper_bound
        data[data<lower_bound]=lower_bound
        # Interpolate missing values
        data_interp = pd.DataFrame(data).interpolate(method='linear', axis=0).values
        im = ax.contourf(lon_grid, lat_grid, data_interp, levels=bounds,
            extent=[lon_grid.min(), lon_grid.max(), lat_grid.min(), lat_grid.max()],
            cmap=cmap, vmin=lower_bound, vmax=upper_bound, zorder=11)

    # add title
    plt.title(title, fontsize=18)
    # add a colorbar
    cbar_ax = fig.add_axes([0.188, -0.04, 0.623, 0.05]) # [left, bottom, width, height]
    num_ticks = 7  # Adjust this value as needed
    tick_locator = ticker.MaxNLocator(nbins=num_ticks)
    tick_formatter = ticker.ScalarFormatter(useMathText=True)
    cbar = plt.colorbar(im, cax=cbar_ax, orientation='horizontal', shrink=1.2,
                        ticks=tick_locator, format=tick_formatter, extend='both')
    #plt.subplots_adjust(top=0.95, bottom=0.15, left=0.0, right=1, hspace=0.2, wspace=0.2)
    cbar.ax.tick_params(labelsize=12)
    cbar.set_label(label, fontsize=14)
    if save=='yes':
        fig.savefig('figures/my_figure.png', dpi=300, bbox_inches='tight')
    plt.tight_layout()
    plt.show()


In [ ]:
import xarray as xr
import numpy as np
spi_ds = xr.open_dataset('SPEI12.nc')

In [ ]:
# Specify the month and year of interest
target_month = 8
target_year = 1992
spi_time = spi_ds['time']
delta_time = abs((spi_time.dt.month - target_month) + (spi_time.dt.year - target_year) * 12)
t = delta_time.argmin()
#t = 309
month = spi_ds.time.values[t].astype('datetime64[M]').astype(int) % 12 + 1
year = spi_ds.time.values[t].astype('datetime64[Y]').astype(int) + 1970
#def plot_map(interp_temps, title, label, lower_bound, upper_bound, vcenter, option, cmap_type, save, filename):
plot_map(spi_ds.SPEI.values[t,:], 'SPEI12', str(month)+'-'+str(year), -3, 3, 0, 'contourf', 'SPI_cmap', 'no', 'noname')


In [ ]:
def find_droughts(time, spi_point):
    # initialize output arrays
    dry_months = np.zeros_like(spi_point)
    duration = np.zeros_like(spi_point)
    intensity = np.zeros_like(spi_point)
    frequency = np.zeros_like(spi_point)
    starts = np.zeros_like(spi_point)
    # find droughts
    drought_start = -1
    num_droughts = 0
    num_extreme_droughts = 0 # new counter for extreme droughts
    current_year = int(str(time[0].astype('M8[Y]')).split('-')[0])
    for i, (spi, dt) in enumerate(zip(spi_point, time)):
        year = int(str(dt.astype('M8[Y]')).split('-')[0])
        if year > current_year:
            current_year = year
            num_droughts = 0
            num_extreme_droughts = 0 # reset counters at the start of each new year

        #if spi < 0: # old drought definition
        if spi <=-1:
            if drought_start == -1:
                drought_start = i
            dry_months[i] = 1
        else:
            if drought_start != -1:
                drought_end = i - 1
                drought_duration = drought_end - drought_start + 1
                duration[drought_start] = drought_duration
                starts[drought_start] = 1
                drought_intensity = np.sum(spi_point[drought_start:drought_end+1])
                intensity[drought_start] = drought_intensity
                drought_peak = np.min(spi_point[drought_start:drought_end+1])
                if drought_peak<=0: # # count all droughts
                    frequency[drought_start] = num_extreme_droughts + 1 # increment the frequency counter
                    num_extreme_droughts += 1 # increment the counter for extreme droughts only
                num_droughts += 1 # always increment the total drought counter
                drought_start = -1

    # handle last drought if it exists
    if drought_start != -1:
        drought_end = len(spi_point) - 1
        drought_duration = drought_end - drought_start + 1
        duration[drought_start] = drought_duration
        starts[drought_start] = 1
        drought_intensity = np.sum(spi_point[drought_start:drought_end+1])
        intensity[drought_start] = drought_intensity
        drought_peak = np.min(spi_point[drought_start:drought_end+1])
        if drought_peak<=0: # count all droughts
            frequency[drought_start] = num_extreme_droughts + 1 # increment the frequency counter
            num_extreme_droughts += 1 # increment the counter for extreme droughts only

    return duration, intensity, dry_months, frequency, starts

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

# Compute drought intensity, duration and frequency at all points and save results
spi_ds = xr.open_dataset('SPEI12.nc')
lat_grid = np.linspace(55.3376, 69.1763, 100) 
lon_grid = np.linspace(11, 24.113, 100)

# Create empty dataset with desired dimensions and coordinates
ds = xr.Dataset(
    {
        'dr_durs': (['time', 'lat', 'lon'], np.zeros((len(spi_ds.time.values), 100, 100))),
        'dr_ints': (['time', 'lat', 'lon'], np.zeros((len(spi_ds.time.values), 100, 100))),
        'dr_mons': (['time', 'lat', 'lon'], np.zeros((len(spi_ds.time.values), 100, 100))),
        'dr_freq': (['time', 'lat', 'lon'], np.zeros((len(spi_ds.time.values), 100, 100))),
        'dr_star': (['time', 'lat', 'lon'], np.zeros((len(spi_ds.time.values), 100, 100))),
    },
    coords={
        'time': spi_ds.time.values,
        'lat': lat_grid,
        'lon': lon_grid,
    }
)

# Fill in dataset with drought data
for i in range(len(lat_grid)):
    for j in range(len(lon_grid)):
        print(i,j)
        time = spi_ds.sel(lat=lat_grid[i], lon=lon_grid[j])['time'].values
        spi_values = spi_ds.sel(lat=lat_grid[i], lon=lon_grid[j])['SPEI'].values
        duration, intensity, dry_months, frequency, starts = find_droughts(time, spi_values)
        ds['dr_durs'][:,i,j] = duration
        ds['dr_ints'][:,i,j] = intensity
        ds['dr_mons'][:,i,j] = dry_months
        ds['dr_freq'][:,i,j] = frequency
        ds['dr_star'][:,i,j] = starts

# Save dataset as netCDF file
ds.to_netcdf('drought_data_SPEI12.nc')
ds.close()

In [ ]:
import xarray as xr
dr_ds = xr.open_dataset('drought_data_SPEI12.nc')
dr_ds = dr_ds.sel(time=slice('1923-01-01', None)) # Use for SPEI

In [ ]:
import numpy as np
#np.nanmax(dr_ds.dr_durs.values)
#max_dur = np.nanmax(dr_ds.dr_durs.values, axis=0) # maximum drought durations
#max_int = np.nanmin(dr_ds.dr_ints.values, axis=0) # strongest drought intensities
#tot_num = dr_ds.dr_nums.values
# define a function to calculate percentiles along the time dimension
def calc_percentiles(data):
    return np.nanpercentile(data, [10, 50, 90], axis=0)
# convert xarray data to a numpy array
#dr_peak_np = spi_ds.SPEI.values
#dr_peak_np[dr_peak_np==0]=np.nan
#dr_durs_np = dr_ds.dr_durs.values
dr_durs_np = dr_ds_decade.dr_durs.values # changed to total duration of droughts per decade
#dr_durs_np[dr_durs_np==0]=np.nan
dr_ints_np = dr_ds_decade.dr_ints.values
#dr_ints_np[dr_ints_np==0]=np.nan
#dr_freq_np = dr_ds_yearly.dr_freq.values # number of droughts per year
#dr_freq_np = dr_ds_yearly.dr_mons.values # number of dry months per year
#dr_freq_np = dr_ds_decade.dr_mons.values # number of dry months per decade
#dr_freq_np = dr_ds_yearly.dr_star.values # number of droughts per year
#dr_freq_np = dr_ds_decade2.dr_freq.values # number of droughts per decade
dr_freq_np = dr_ds_decade.dr_star.values # number of droughts per decade
#dr_freq_np[dr_freq_np==0]=np.nan
#dr_freq_np = np.nan_to_num(dr_freq_np, nan=0)
# calculate percentiles using the function and numpy array
#peak_perc_np = calc_percentiles(dr_peak_np)
dur_perc_np = calc_percentiles(dr_durs_np)
int_perc_np = calc_percentiles(dr_ints_np)
freq_perc_np = calc_percentiles(dr_freq_np)



In [ ]:

# Plot strongest intensity
plt.rcdefaults()
#plot_data=np.nanmin(dr_ints_np, axis=0)
plot_data=int_perc_np[0]*(-1) # absolute values
lb=int(np.percentile(plot_data[plot_data>0], 0.01))
ub=int(np.percentile(plot_data[plot_data>0], 99.99))
vcenter=int(np.percentile(plot_data[plot_data>0], 50))
ub=14
lb=6
#vcenter=-300
plot_map(plot_data, 'Drought Intensity Hotspots', 'Top decile of annual cumulative drought intensity', lb, ub, vcenter, 'imshow', 'normal','no','no')

In [ ]:
ub

In [ ]:
# Plot longest duration
#plot_data=np.nanmax(dr_durs_np, axis=0)/12
plot_data=dur_perc_np[2]
lb=int(np.percentile(plot_data[plot_data>0], 5))
ub=int(np.percentile(plot_data[plot_data>0], 95))
#vcenter=int(np.percentile(plot_data[plot_data>0], 50))
#ub=9
#lb=4
#vcenter=5
plot_map(plot_data, 'Drought Duration Hotspots', 'Top decile of decadal cumulative drought duration (months)', lb, ub, vcenter, 'imshow', 'normal','no','no')

In [ ]:
# Reset Matplotlib settings to defaults
#plt.rcParams.update(plt.rcParamsDefault)
plot_data=freq_perc_np[2] # 90th percentile of droughts per decade or year depending on previous cell
lb=int(np.percentile(plot_data[plot_data>0], 5))
ub=int(np.percentile(plot_data[plot_data>0], 95))
vcenter=int(np.percentile(plot_data[plot_data>0], 50))
#ub=10
#lb=4
#vcenter=2
#plot_map(plot_data, '90th Percentile Dry Month Count per Decade', lb, ub, vcenter, 'imshow', 'normal','no','no')
plot_map(plot_data, 'Drought Frequency Hotspots', 'Top decile of decadal count of droughts', lb, ub, vcenter, 'imshow', 'normal','no','no')
#plot_map(plot_data, 'Drought Frequency (Dry Months per Decade)', '90th Percentile Values', lb, ub, vcenter, 'imshow', 'normal','no','no')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
ds=dr_ds_yearly
# Create a function to plot scatter plots
def plot_scatter(x, y, xlabel, ylabel, title):
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=x, y=y)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

# Create the scatter plots
plot_scatter(ds['dr_durs'].values.flatten(), ds['dr_freq'].values.flatten(), 
             'Duration', 'Frequency', 'Duration vs Frequency')
plot_scatter(ds['dr_ints'].values.flatten(), ds['dr_freq'].values.flatten(), 
             'Intensity', 'Frequency', 'Intensity vs Frequency')
plot_scatter(ds['dr_ints'].values.flatten(), ds['dr_durs'].values.flatten(), 
             'Intensity', 'Duration', 'Intensity vs Duration')

In [ ]:
import seaborn as sns
from scipy.stats import pearsonr
def plot_reg(df, colx, coly, lbx, lby):
    df=df.dropna()
    # Calculate correlation coefficient
    corr, _ = pearsonr(df[colx], df[coly])
    # Create scatter plot with regression line and correlation coefficient
    g=sns.jointplot(x=colx, y=coly, data=df, kind='reg',  line_kws={"color":"red"})
    # Add text box with correlation coefficient
    g.ax_joint.text(0.1, 0.9, f"R = {corr:.2f}", fontsize=12,
                transform=g.ax_joint.transAxes,
                bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))
    # Set custom x and y axis labels
    g.set_axis_labels(lbx, lby)
    # Display the plot
    plt.show()    

In [ ]:
ssi_ds = xr.open_dataset('SSI12.nc')
ssi_df=ssi_ds.to_dataframe()

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr, linregress

def calculate_trends(data_array, alpha=0.05):
    timesteps, n_lat, n_lon = data_array.shape

    x = np.arange(timesteps)
    rate_of_change = np.empty((n_lat, n_lon))

    for i in range(n_lat):
        for j in range(n_lon):
            y = data_array[:, i, j]
            mask = ~np.isnan(y)
            if mask.sum() > 1:
                slope, intercept, r_value, p_value, std_err = linregress(x[mask], y[mask])
                if p_value <= alpha:
                    rate_of_change[i, j] = slope
                else:
                    rate_of_change[i, j] = np.nan
            else:
                rate_of_change[i, j] = np.nan

    return rate_of_change

In [ ]:
dr_ds = xr.open_dataset('drought_data_SPEI12.nc')
dr_ds = dr_ds.sel(time=slice('1923-01-01', None)) # Use for SPEI
dr_ds_yearly =dr_ds.resample(time='1Y').sum('time')

In [ ]:
dr_dur_rate=calculate_trends(dr_durs_np) # Total duration per decade
plot_map(dr_dur_rate, 'Drought Duration Trends', 'Annual change of cumulative drought duration (months)', -0.1, 0.1, 0, 'imshow','coolwarm', 'no', 'no')

In [ ]:
#dr_freq_corr=calculate_trends(dr_ds.dr_freq) 
dr_freq_rate=calculate_trends(dr_freq_np)
plot_map(dr_freq_rate, 'Drought Frequency Trends', 'Decadal change of drought count', -1.7, 1.7, 0, 'imshow','coolwarm', 'no', 'no')

In [ ]:
#%%capture --no-display
dr_int_rate=calculate_trends(dr_ints_np*(-1)) # Total drought intensity per year, -1 for positive intensities
plot_map(dr_int_rate, 'Drought Intensity Trends', 'Decadal change of cumulative drought intensity', -0.1, 0.1, 0, 'imshow','coolwarm', 'no', 'no')
#plot_map(dr_int_corr, 'Drought Intensity Trends (R-value)', -0.5, 0.5, 0, 'imshow', 'normal','no','no')